# NER Pipeline

...
## 📄 Overview

...


---

## 🛠 Dependencies

To run this notebook, make sure you have the following:

### ✅ Requirements

- **Python 3.8 or higher**
- **Jupyter Notebook** or **JupyterLab** (to run `.ipynb` files)

Install Jupyter Notebook:
```bash
pip install notebook
```
### 📦 Python Packages
```bash
python -m spacy download fr_core_news_sm
python -m spacy download fr_core_news_md
python -m spacy download fr_core_news_lg

```
---

In [7]:
# -------------- IMPORTS ----------------
import time
import pandas as pd
import spacy
import numpy as np
from spacy import displacy
from time import sleep
import bs4
from bs4 import BeautifulSoup
import os
import re
import json
from pathlib import Path
# ---------------------------------------

# Load The Data

In [ ]:
def load_excel_file(file_path:str):


    """
        Load a Excel file into a DataFrame

        Parameters:
            file_path (str): The file path

        Returns:
            df (pd.Dataframe) : the result of Excel file
    """

    # Start Chrono
    start_chrono = time.time()

    # Open the Excel file
    df = pd.read_excel(file_path)

    # Clean
    df.drop(df.columns[0], axis=1, inplace=True) # Drop the Column
    #print(df.isnull().sum())
    df = df.fillna('') # clean the missing description
    
    # End Chrono
    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Load data in : {chrono:.2f}s")

    return df

#df = load_excel_file("ressources/20231101_raw.xlsx")

Load data in : 2.11s


# Use Spacy 

In [ ]:
# Choose the Spacy modele 
#NER = spacy.load("fr_core_news_sm")

In [21]:

def apply_spacy(df, NER):
    """
        Apply Spacy

        Parameters:
            df (pd.Dataframe): the dataframe 
    """

    ner_spacy = [] # ("text_entite", "label")
    ner_spacy_id = [] # ("start", "end")

    # Start Chrono
    start_chrono = time.time()
    # foreach row , use spacy on the description
    for idx, row in df.iterrows():
        desc = str(row["desc"]) # extract the description
        entities = NER(desc) # use spacy 
        entities_founds = set() # Manage duplicates automatically
        entities_locations = []

        for word in entities.ents:
            entities_founds.add((word.text, word.label_))
            entities_locations.append((word.start, word.end))

        ner_spacy.append(entities_founds)
        ner_spacy_id.append(entities_locations)

    # End Chrono
    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Apply Spacy in : {chrono:.2f}s")

    return (ner_spacy, ner_spacy_id)

# ner_spacy, ner_spacy_id = apply_spacy(df)

In [22]:
def build_ner_dataframe(df, ner_spacy, ner_spacy_id, NER):
    """
    Build a new DataFrame from the original DataFrame and precomputed NER outputs.
    
    The original DataFrame must contain at least two columns: 'titles' and 'desc'.
    For each row, the function uses the provided NER results (entity text/label and their corresponding indices)
    to extract a portion of the description text with some context around each entity.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame containing a 'titles' column and a 'desc' (description) column.
        ner_spacy (list): A list (one per row) of lists of tuples, each tuple containing the entity text and its label.
                          For example: [[("Barack Obama", "PERSON"), ("Washington", "GPE")], ...]
        ner_spacy_id (list): A list (one per row) of lists of tuples, each tuple containing the start and end indices of the entity.
                             For example: [[(0, 12), (20, 30)], ...]
                             
    Returns:
        pd.DataFrame: A new DataFrame with the following columns:
                      - "titles": the original title repeated for each extracted entity.
                      - "NER": the extracted entity text.
                      - "NER_label": the label/type of the entity.
                      - "desc": a snippet of the original description around the entity.
    """

    start_chrono = time.time()

    # Convert the 'titles' and 'desc' columns to lists
    titles_list = df["titles"].tolist()
    descriptions_list = df["desc"].tolist()
    
    # Initialize lists to store the new data
    title_list = []
    text_list = []
    label_list = []
    location_list = []
    file_id_list = []
    
    # Loop through each entry in the original DataFrame (by index)
    for i in range(len(titles_list)):
        current_desc = descriptions_list[i]
        # Process the description using the NER function to obtain a full processed document.
        # Assume that NER() returns an object where converting it to a string gives its full text.
        current_NER_doc = NER(current_desc)
        
        # Get the precomputed NER entities and their indices for this row
        current_NER = ner_spacy[i]
        indices = ner_spacy_id[i]
        
        # Get the title associated with the current row
        current_title = titles_list[i]
        
        # Loop over each entity and its corresponding indices
        for entity, idx in zip(current_NER, indices):
            entity_text, entity_label = entity
            
            # Append data to the corresponding list
            title_list.append(current_title)
            text_list.append(entity_text)
            label_list.append(entity_label)
            file_id_list.append(i)
            
            # Extract a snippet from the full document text around the entity for context.
            # If the entity starts near the beginning, we extract from the start.
            if idx[0] < 10:
                # If entity is close to the end of the document, include all text until the end.
                if idx[1] + 8 > len(current_NER_doc):
                    location_list.append(current_NER_doc)
                else:
                    location_list.append(current_NER_doc[:idx[1] + 8])
            else:
                # Else extract 10 characters before the entity start.
                # If the entity is near the end, extract until the end.
                if idx[1] + 8 > len(current_NER_doc):
                    location_list.append(current_NER_doc[idx[0] - 10:])
                else:
                    location_list.append(current_NER_doc[idx[0] - 10: idx[1] + 8])
    
    # Build a new DataFrame using the collected lists.
    new_df = pd.DataFrame(zip(title_list, text_list, label_list, location_list, file_id_list))
    
    # Rename the columns to be more informative.
    new_df.rename(columns={0: "titles", 1: "NER", 2: "NER_label", 3: "desc", 4:"file_id"}, inplace=True)

    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Generate Spacy DataFrame in : {chrono:.2f}s")
    
    return new_df

# df_ner = build_ner_dataframe(df, ner_spacy, ner_spacy_id)
# df_ner.head()
#df_ner.to_excel("Result/xlsx/DataFrame_Spacy.xlsx")

# CasEN

In [ ]:
def generate_text_files(df):
    start_chrono = time.time()

    # Generate text file 
    text_location = "Result/Corpus/All.txt"
    
    with open(text_location, 'w', encoding="utf-8") as f:
        for idx, desc in enumerate(df["desc"]):
            f.write(f'<doc id="{idx}">')
            f.write(str(desc))
            f.write('</doc>\n')

    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Generate text files in : {chrono:.2f}s")

#generate_text_files()

In [24]:
# Run CasEN
# casen_ipynb_path = "../CasEN_fr.2.0/CasEN.ipynb"
# get_ipython().run_line_magic('run', str(casen_ipynb_path))

In [75]:
def extract_entities_from_tree(soup_doc: BeautifulSoup, doc_id: int):
    """
    Ne remonte que les entités ayant un grf et sans ancêtre grf,
    et stocke pour chacune :
      - file_id, tag, text, grf (main)
      - second_graph  = grf du 1er enfant direct
      - third_graph   = grf du 2ème enfant direct
    """
    entities = []
    excluded = {"s", "p", "doc"}

    # On parcourt tous les éléments taggés avec grf
    for element in soup_doc.find_all(lambda tag: tag.name not in excluded and tag.has_attr("grf")):
        # Si un des ancêtres a aussi un grf, on l'ignore (c'est un sous-élément)
        if any(ancestor.has_attr("grf") for ancestor in element.parents if ancestor.name not in excluded):
            continue

        main_grf = element["grf"]
        # on collecte les grf des enfants directs pour second et third
        children = [child for child in element.find_all(recursive=False) if child.has_attr("grf")]
        second = children[0]["grf"] if len(children) >= 1 else ""
        third  = children[1]["grf"] if len(children) >= 2 else ""

        entities.append({
            "file_id":       doc_id,
            "tag":           element.name,
            "text":          element.get_text(),
            "grf":           main_grf,
            "second_graph":  second,
            "third_graph":   third
        })

    return entities

def extract_entities_from_file(result_path :str):

    with open(result_path, 'r', encoding="utf-8") as f:
        content = f.read()

    content = re.sub(r'</?s\b[^>]*>', '', content)
    soup = BeautifulSoup(content, 'html.parser')

    all_entities = []

    for doc in soup.find_all('doc'):
        doc_id = int(doc.attrs.get("id"))
        ents = extract_entities_from_tree(doc, doc_id)
        all_entities.extend(ents)

    return all_entities

def casEN_df_single(df: pd.DataFrame) -> pd.DataFrame:
    """
    Build a DataFrame of NER results from a single CasEN output file.
    """
    start = time.time()

    list_entities = extract_entities_from_file("Result/CasEN_Result/Res_CasEN_Analyse_synthese_grf/All.result.txt")
    rows = []
    for entity in list_entities:
        idx = entity["file_id"]
        ner_text = entity["text"]
        context = get_ner_context(df.loc[idx, "desc"], ner_text, window=7)
        rows.append({
            "titles": df.loc[idx, "titles"],
            "NER": ner_text,
            "NER_label": find_ner_label(entity["tag"]),
            "desc": context,
            "method": "CasEN",
            "main_graph": entity["grf"],
            "second_graph": entity.get("second_graph", ""),
            "third_graph": entity.get("third_graph", ""),
            "file_id": idx
        })
    df_casen = pd.DataFrame(rows)
    print(f"Built CasEN DataFrame in {time.time() - start:.2f}s")
    return df_casen

In [8]:
def get_ner_context(full_desc: str, ner_text: str, window: int = 10) -> str:
    """
    Extracts a context window around the first match of the NER text in the full descriptionE
    
    Parameters:
        full_desc (str): The full text description.
        ner_text (str): The NER text to find in the description.
        window (int): Number of words to extract before and after the match

    Returns:
        str: A snippet of the description centered around the NER match
    """
    # Normalize input for search
    full_desc_clean = re.sub(r'[^\w\s]', '', full_desc.lower())
    ner_clean = re.sub(r'[^\w\s]', '', ner_text.lower())

    # Tokenize words
    words = full_desc.split()
    clean_words = full_desc_clean.split()
    ner_words = ner_clean.split()

    # Try to find full match of all NER words
    for i in range(len(clean_words) - len(ner_words) + 1):
        if clean_words[i:i+len(ner_words)] == ner_words:
            start = max(0, i - window)
            end = min(len(words), i + len(ner_words) + window)
            return ' '.join(words[start:end])

    return full_desc

def find_ner_label(tag:str) -> str:
    if tag in ["persname", "surname", "forename"]:
        return "PER"
    elif tag in ['placename','geoname', 'adress', 'adrline']:
        return "LOC"
    elif tag in ["orgname", "org"]:
        return "ORG"
    else:
        return "MISC"

def casEN_df(df:pd.DataFrame) -> pd.DataFrame:
    """
        Extracts named entities from external CasEN result files and compiles them into a DataFrame.

        Parameters:
            df (pd.DataFrame) : DataFrame containing original data 
        
        Returns:
            pd.DataFrame -> result of CasEN analyse in DataFrame format
    
    """
    start_chrono = time.time()

    titles = df["titles"].to_list()
    desc = df["desc"].to_list()

    # Make a new DataFrame 
    df_casEN = pd.DataFrame(columns=["titles", "NER", "NER_label", "desc", "method", "main_graph", "second_graph", "third_graph", "file_id"])

    for index in range(len(titles)):

        entities = extract_entities_from_file(f"Result/CasEN_Result/Res_CasEN_Analyse_synthese_grf/np{index}.result.txt")

        for entity in entities:

            # Find every graph
            main_graph = entity.get("grf", "")
            secondary_graph = [value["grf"] for value in entity.values() if isinstance(value, dict) and "grf" in value]
            # Make desc shorter
            ner_text = entity.get("text", "")
            context_desc = get_ner_context(desc[index], ner_text, 7)

            row = {
                "titles" : titles[index],
                "NER" : ner_text,
                "NER_label" : find_ner_label(entity.get("tag")),
                "desc" : context_desc,
                "method" : "CasEN",
                "main_graph" : main_graph,
                "second_graph" : secondary_graph[0] if len(secondary_graph) > 0 else "",
                "third_graph" : secondary_graph[1] if len(secondary_graph) > 1 else "",
                "file_id" : index
            }
            df_casEN.loc[len(df_casEN)] = row

    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Generate CasEN DataFrame in : {chrono:.2f}s")

    return df_casEN
        
#df_casEN = casEN_df_single(df)
#df_casEN.to_excel("Result/xlsx/DataFrame_CasEN.xlsx")

In [28]:
def determine_method(row):
    if row['_merge'] == 'both':
        return "intersection"
    elif row['_merge'] == 'left_only':
        return row['method_spacy']
    elif row['_merge'] == 'right_only':
        return row['method_casEN']
    else:
        return "Spacy"

def merge_spacy_casEN(df_spacy:pd.DataFrame,df_casEN:pd.DataFrame) -> pd.DataFrame:
    """

    
    """
    start_chrono = time.time()

    if "method" not in df_spacy.columns:
        df_spacy["method"] = "Spacy"

    # Create unique key on ("titles", "NER", "NER_label")
    df_spacy["key"] = df_spacy[["titles", "NER", "NER_label", "file_id"]].apply(lambda x: tuple(x), axis=1)
    df_casEN["key"] = df_casEN[["titles", "NER", "NER_label", "file_id"]].apply(lambda x: tuple(x), axis=1)

    # merge dataframs on the key
    merge = pd.merge(df_spacy, df_casEN, on='key', how='outer',suffixes=('_spacy', '_casEN'), indicator=True)

    # fix the shared columns
    merge["titles"] = merge["titles_spacy"].combine_first(merge["titles_casEN"])
    merge['method'] = merge.apply(determine_method, axis=1)
    merge["NER"] = merge["NER_spacy"].combine_first(merge["NER_casEN"])
    merge["NER_label"] = merge["NER_label_spacy"].combine_first(merge["NER_label_casEN"])
    merge["desc"] = merge["desc_casEN"].combine_first(merge["desc_spacy"])
    merge["file_id"] = merge["file_id_casEN"].combine_first(merge["file_id_spacy"])
    
    merge = merge.sort_values(
        by=["titles"], 
        ascending=[False]
    ).reset_index(drop=True)

    merge["manual cat"] = ""
    merge["extent"] = ""
    merge["correct"] = ""
    merge["category"] = ""

    final_columns = ['manual cat', 'correct', 'extent', 'category','titles', 'NER', 'NER_label', 'desc', 'method',
                     'main_graph', 'second_graph', 'third_graph', "file_id"]


    merge = merge.drop_duplicates(subset=["titles", "NER", "NER_label", "method", "main_graph", "second_graph", "third_graph"])

    end_chrono = time.time()
    chrono = end_chrono - start_chrono
    print(f"Merge CasEN & Spacy in : {chrono:.2f}s")

    return merge[final_columns]

# intersection = merge_spacy_casEN(df_ner, df_casEN)
# intersection.head()

In [29]:
# Converte into Excel File
#intersection.to_excel("Result/xlsx/intersection.xlsx", index=False)

In [ ]:
'''
spacy model :fr_core_news_sm
Load data in : 1.42s
Apply Spacy in : 75.02s
Generate Spacy DataFrame in : 63.34s
Generate text files in : 7.67s
Generate CasEN DataFrame in : 81.52s
Merge CasEN & Spacy in : 0.86s
Total Execute in : 960.58s

CasEN : 730.75s

'''

'\nspacy model :fr_core_news_sm\nLoad data in : 1.42s\nApply Spacy in : 75.02s\nGenerate Spacy DataFrame in : 63.34s\nGenerate text files in : 7.67s\nGenerate CasEN DataFrame in : 81.52s\nMerge CasEN & Spacy in : 0.86s\nTotal Execute in : 960.58s\n\nCasEN : 265.93s\n\n'

In [3]:
import Pipeline_upgrade

In [4]:
# Spacy modeles
small = "fr_core_news_sm"
medium = "fr_core_news_md"
large = "fr_core_news_lg"

NER = spacy.load(small)
data = "ressources/20231101_raw.xlsx"
Pipeline_upgrade.Pipeline(data, "Result/xlsx/intersection.xlsx")

spacy model :fr_core_news_sm
Load data in : 5.51s
Apply Spacy in : 71.66s
Generate Spacy DataFrame in : 59.88s
Generate text files in : 0.02s
All.txt
C:\Users\valen\Documents\Informatique-L3\Stage\Stage\Result\Corpus		 -> results in : C:\Users\valen\Documents\Informatique-L3\Stage\Stage\Result\CasEN_Result\Res_CasEN_Analyse_synthese_grf
1 files to process with CasEN in  C:\Users\valen\Documents\Informatique-L3\Stage\Stage\Result\Corpus
C:\Users\valen\AppData\Local\Unitex-GramLab\App\UnitexToolLogger.exe { BatchRunScript -e -i "C:\Users\valen\Documents\Informatique-L3\Stage\Stage\Result\Corpus" -o "C:\Users\valen\Documents\Informatique-L3\Stage\Stage\Result\CasEN_Result\Res_CasEN_Analyse_synthese_grf" -t1 "C:\Users\valen\Documents\Informatique-L3\Stage\CasEN_fr.2.0\my_CasEN_lingpkg.zip" -f -s "script\CasEN_Analyse_synthese_grf.uniscript" }
Built CasEN DataFrame in 3.20s
Merge CasEN & Spacy in : 0.69s
Total Execute in : 395.67s


In [11]:
"""
spacy model :fr_core_news_sm
Load data in : 2.19s
Apply Spacy in : 75.84s
Generate Spacy DataFrame in : 61.66s
Generate text files in : 0.02s
Built CasEN DataFrame in 2.78s
Merge CasEN & Spacy in : 0.50s
Total Execute in : 408.92s

CasEN : 265.93s

"""

'\nspacy model :fr_core_news_sm\nLoad data in : 2.19s\nApply Spacy in : 75.84s\nGenerate Spacy DataFrame in : 61.66s\nGenerate text files in : 0.02s\nBuilt CasEN DataFrame in 2.78s\nMerge CasEN & Spacy in : 0.50s\nTotal Execute in : 408.92s\n\nCasEN : 265.93s\n\n'

In [12]:
"""
    PB : en gardant que les "graphes" les plus grands, si jamais casEN trouve le prenom/nom et spacy que le nom alors il n'y a plus d'intersection
        sur le nom.

        

"""

'\n    PB : en gardant que les "graphes" les plus grands, si jamais casEN trouve le prenom/nom et spacy que le nom alors il n\'y a plus d\'intersection\n        sur le nom.\n\n        \n\n'

In [ ]:
# Correct the Excel file
def correct_excel(df1 : pd.DataFrame, df2:pd.DataFrame):

    df1 = df1.copy()
    df2 = df2.copy()

    # Créer les clés dans les deux DataFrames
    df1["key"] = df1[["titles", "NER", "NER_label", "method", "hash"]].apply(tuple, axis=1)
    df2["key"] = df2[["titles", "NER", "NER_label", "method", "file_id"]].apply(tuple, axis=1)

    # Colonnes à transférer
    cols_to_copy = ["manual cat", "correct", "extent", "category"]

    # Créer un dictionnaire : clé -> valeurs des colonnes à copier
    df1_dict = df1.set_index("key")[cols_to_copy].to_dict(orient="index")

    # Pour chaque ligne de df2, copier si la clé existe
    for col in cols_to_copy:
        df2[col] = df2["key"].map(lambda k: df1_dict.get(k, {}).get(col, ""))

    # Supprimer la clé temporaire
    df2.drop(columns=["key"], inplace=True)

    return df2


correction = pd.read_excel("ressources/20231101_Digital3D_Tele-Loisirs _telerama_NER_weekday_evaluation_v3(1).xlsx")
df_pipeline = pd.read_excel("Result/xlsx/intersection.xlsx")

corrected_pipeline = correct_excel(correction, df_pipeline)
corrected_pipeline.to_excel("Result/xlsx/Final_result2.xlsx", index=False)

In [6]:
print(spacy.__version__)

3.8.5
